In [2]:
! pip install datasets

^C


In [3]:
from datasets import load_dataset

# Load Career Guidance Reddit Dataset
dataset = load_dataset("mb7419/career-guidance-reddit")

# Print an example
print(dataset["train"][0])

Using the latest cached version of the dataset since mb7419/career-guidance-reddit couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\DELL\.cache\huggingface\datasets\mb7419___career-guidance-reddit\default\0.0.0\27a7fb49e3bb94b42587a93482c80c97dff3ed34 (last modified on Wed Feb 19 15:35:40 2025).


{'id': 'd1qr9r', 'title': 'How old were you when you knew what you wanted to do?', 'body': 'I think it s so dumb that we force 18 year olds to know what they want to do for the rest of their lives. I m 28 and I only just figured out what I wanted to do like 4 years ago.   How old were you and what did you end up doing?', 'created_utc': '2019-09-09 13:15:22', 'url': 'https://www.reddit.com/r/careerguidance/comments/d1qr9r/how_old_were_you_when_you_knew_what_you_wanted_to/', 'retrieved_on': '2020-04-04 19:48:42', 'question_content': 'how old were you when you knew what you wanted to do  i think it s so dumb that we force 18 year olds to know what they want to do for the rest of their lives  i m 28 and i only just figured out what i wanted to do like 4 years ago    how old were you and what did you end up doing ', 'dominant_topic': 0, 'dominant_topic_name': 'Career Exploration and Uncertainty', '__index_level_0__': 6462}


In [ ]:
# ! pip install transformers

  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.2-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
Using cached safetensors-0.5.2-cp38-abi3-win_amd64.whl (303 kB)
Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl (2.4 MB)


In [ ]:
# ! pip install torch
# ! pip install faiss-cpu


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pytorch
Failed to build pytorch


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\DELL\AppData\Local\Temp\pip-install-3439g9gz\pytorch_70059b2042fa436cab31e2880198eca7\setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pytorch)


In [5]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np




# Load BERT tokenizer & model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, force_download=True)

# Function to generate embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Create FAISS Index for fast search
d = 768  # BERT embedding size
faiss_index = faiss.IndexFlatL2(d)

# Store embeddings & questions
career_questions = []
career_answers = []
career_vectors = []

for entry in dataset["train"]:
    question = entry["title"]
    answer = entry["body"]
    embedding = get_embedding(question)

    career_questions.append(question)
    career_answers.append(answer)
    career_vectors.append(embedding)

# Convert to FAISS format
career_vectors = np.vstack(career_vectors)
faiss_index.add(career_vectors)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
def retrieve_career_advice(user_query):
    user_embedding = get_embedding(user_query)
    _, idx = faiss_index.search(user_embedding, 1)  # Find closest match
    best_match_index = idx[0][0]
    
    return career_answers[best_match_index]

# Example Usage
user_input = "What skills do I need to be a software engineer?"
response = retrieve_career_advice(user_input)
print("Career Advice:", response)


Career Advice: I m currently in online learning to obtain my Medical &amp  Coding License  however  I ve always had a passion for Technology. I decided to browse online and found the following link. Everything within the program overview fascinates me and I feel like this could be my calling  however  I just want to have a solid idea of what is required or what I need to obtain a job within this career field. I ve already requested information online and I wanted to ask my fellow Redditors for their support. Any help  advice  or wisdom would be much appreciated.


In [ ]:
! pip install sentencepiece


   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   ---------------------------------------- 992.0/992.0 kB 5.8 MB/s eta 0:00:00


In [9]:
from transformers import LlamaForCausalLM, LlamaTokenizer

# Load LLaMA 2 model & tokenizer
model_name = "meta-llama/Llama-2-7b-hf"
llama_tokenizer = LlamaTokenizer.from_pretrained(model_name)
llama_model = LlamaForCausalLM.from_pretrained(model_name, device_map="auto")

# Function to generate LLaMA-based responses
def generate_llama_response(user_query):
    input_ids = llama_tokenizer.encode(user_query, return_tensors="pt").to("cuda")
    output_ids = llama_model.generate(input_ids, max_length=100)
    response = llama_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return response


OSError: Can't load tokenizer for 'meta-llama/Llama-2-7b-hf'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Llama-2-7b-hf' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.

In [8]:
! huggingface-cli login


^C
